# 4.2 손실 함수

신경망 학습에서는 현재의 상태를 '하나의지표'로 표현  
지표를 가장 좋게 만들어 주는 가중치 매개변수의 값을 탐색  
신경망 학습에서 사용하는 지표는 **손실 함수(loss function)**라고 한다.

### 대표적인 손실함수

4.2 손실 함수

신경망 학습에서는 현재의 상태를 '하나의지표'로 표현  
지표를 가장 좋게 만들어 주는 가중치 매개변수의 값을 탐색  
신경망 학습에서 사용하는 지표는 **손실 함수(loss function)**라고 한다.

### 대표적인 손실함수

    - 평균 제곱 오차 (MSE) -> 회귀 문제 (regression)  
    - 교차 엔트로피 오차 (CEE) -> 분류 문제 (classification)  

손실함수는 간단히 말하자면 현재의 신경망이 훈련데이터를 얼마나 잘 처리하지 '못'하는지를 나타냄

## 4.2.1 평균 제곱 오차

![image](./image/e_4_1.png)

yk: 신경망의 출력 (신경망이 추정한값)  
tk: 정답 레이블    
k: 데이터 차원 수  
원 핫 인코딩: 한 원소만 1로 하고 나머지는 0으로 나타내는 표기법  

In [1]:
import numpy as np

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0,0.1, 0.0, 0.0]
t =[0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # One Hot Encoding

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

print(mean_squared_error(np.array(y), np.array(t)))
y = [0.1,0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(mean_squared_error(np.array(y), np.array(t)))

0.09750000000000003
0.5975


위 결과 를 보면 첫번째의 y가 오차가 더 적은 것을 알 수 있다.

## 4.2.2 교차 엔트로피 오차

![image](./image/e_4_2.png)

공식의 log는 자연로그 log_e이다.  
yk: 신경망의 출력 (신경망이 추정한값)  
tk: 정답 레이블 (원핫 인코딩의 형식으로 이루어져 있음)

원 핫 인코딩의 형식으로 되어있으므로 실질적으로 정답일 때의 자연로그 값을 계산하는 식이 된다.

<img src = "./image/fig_4_3.png" width="40%">

위와 같이 log 함수는 출력 값이 1에 가까워질수록 0에 가까워지고 0일때 발산한다.  
즉 정답일 때의 출력이 작을 수록 오차가 커진다.  

In [2]:
def cross_entropy_error (y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

#delta는 매우 작은 값으로 만약 log0이 될경우 발산하는 것을 대비해서 값을 넣어줌

In [3]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0,0.1, 0.0, 0.0]
t =[0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # One Hot Encoding

print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1,0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


위 결과를 보면 마찬가지로 첫번째 것의 오차가 더 적다는 것을 알 수 있다.

## 4.2.3 미니배치 학습

모든 훈련데이터에 대한 손실함수 값의 합을 지표로 해야 함  
예를 들어 교차 엔트로피 오차는 다음과 같이 나타낼수 있다.  

<img src = "./image/e_4_3.png">

N: 데이터의 개수  
t_nk: n번째 데이터의 k번째 값

N으로 나누므로써 평균 손실 함수를 구할 수 있음  
평균 손실 함수를 사용하면 데이터 개수와 관련 없이 언제든 통일 지표를 구할 수 있음  
다만 모든 데이터에 대해서 일일히 손실함수를 구하는 것은 쉽지 않기 때문에  
근사치 즉 일부만 골라 학습을 진행한다. 이것을 **미니배치**라고 부른다.  
데이터에서 일부만을 무작위로 골른 일부 데이터를 사용하여 학습하는 것을 미니배치 학습이라고 부른다.  

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (X_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [5]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # np.random.choice 함수로 미니배치 구현하기
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [6]:
print(np.random.choice(60000, 10))

[15774  2961  4675 39731 52024 24904 10665 30341 54579 33467]


## 4.2.4 (배치용)교차 엔트로피 오차 구현하기

In [7]:
# 정답 레이블이 원 핫 인코더 형태일 때

def cross_entrpy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return - np.sum(t*np.log(y)) / batch_size

# reshape(x,y) 함수: x열, y행의 metrics로 바꿔주는 함수, 여기서는 정답 레이블의 크기로 차원을 바꿔주는 역할

In [8]:
# 정답 레이블이 숫자들로 구성되어 있을 때

def cross_entry_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

원 핫 인코딩일 때 t=0인 함수는 교차엔트로피의 오차도 0이므로 계산을 무시할 수 있다.  
즉 정답에 해당하는 신경망만으로 오차의 계산이 가능해진다. 

## 4.2.5 왜 손실함수를 설정하는가?

손실함수를 설정하는 잉유는 최적의 매개변수를 찾는 과정에서 미분을 계산하고  
그 미분 값을 단서로 서서히 매개변수 값을 갱신하기 때문이다.  

정확도를 지표로 삼아 매개변수에 변화를 주면 미분 값이 어느순가 0이 되어 가중치 갱신이 멈추게 된다.  
정확도는 매개변수에 따라 불연속적으로 변화하기 때문이다.  
마치 계단함수와 시그모이드 함수의 차이라고 생각하면 될것아다.  
정리하자면 미분 값이 0이되어 제대로된 학습이 이루어지지 않을 수 있어 손실함수를 지표로 매개변수를 갱신해 나가야 한다.